In [ ]:
import pandas as pd

In [ ]:
zelenogorsk = pd.read_csv('../data/gkh_clear_Zelenogorsk.csv', encoding='utf-8', sep=',')
kansk = pd.read_csv('../data/gkh_clear_Kansk.csv', encoding='utf-8', sep=',')
achinsk = pd.read_csv('../data/gkh_clear_Achinsk.csv', encoding='utf-8', sep=',')

In [ ]:
# Используем две библиотеки для геокодирования:
# 1 - геокодеры из проекта geopy
from geopy.geocoders import Nominatim, Yandex
# 2 - стандартная библиотека веб-запросов (для работы с geocode.xyz)
import requests

# И еще парочку:
# - для ограничения скорости запросов
from geopy.extra.rate_limiter import RateLimiter
# - для визуализации процесса
from tqdm import tqdm

In [ ]:
import yaml

with open("../api_keys/geocoding_api.yml", "r") as f:
    api_keys = yaml.safe_load(f)

In [ ]:
geolocator_osm = Nominatim(user_agent="py")
geolocator_ya = Yandex(user_agent = "py", api_key = api_keys.get("yandex_geocoding_api").get("api_key"))
geocode_osm = RateLimiter(geolocator_osm.geocode, min_delay_seconds=1)
geocode_ya = RateLimiter(geolocator_ya.geocode, min_delay_seconds=1)

In [ ]:
zelenogorsk['addr_osm'] = ''
zelenogorsk['lon_osm'] = ''
zelenogorsk['lat_osm'] = ''
zelenogorsk['addr_ya'] = ''
zelenogorsk['lon_ya'] = ''
zelenogorsk['lat_ya'] = ''

In [ ]:
kansk['addr_osm'] = ''
kansk['lon_osm'] = ''
kansk['lat_osm'] = ''
kansk['addr_ya'] = ''
kansk['lon_ya'] = ''
kansk['lat_ya'] = ''

In [ ]:
achinsk['addr_osm'] = ''
achinsk['lon_osm'] = ''
achinsk['lat_osm'] = ''
achinsk['addr_ya'] = ''
achinsk['lon_ya'] = ''
achinsk['lat_ya'] = ''

In [ ]:
task = achinsk
for index, row in task.iterrows():
    location_osm = geocode_osm(str.replace(row['formalname_city'] + ', ' + str(row['formalname_street']) + ', ' + str(row['house_number']) + str(row['building']) + str(row['block']) + str(row['letter']), 'nan', ''))
    print(str.replace(row['formalname_city'] + ', ' + str(row['formalname_street']) + ', ' + str(row['house_number']) + str(row['building']) + str(row['block']) + str(row['letter']), 'nan', ''))
    print('osm: ' + str(location_osm))
    # location_ya = geocode_ya(row['address'])
    # print('ya: ' + str(location_ya))
    if location_osm:
        task.loc[index, 'addr_osm'] = location_osm.address
        task.loc[index, 'lon_osm'] = location_osm.longitude
        task.loc[index, 'lat_osm'] = location_osm.latitude
        # print(location_osm.address)
        # print(str(location_osm.longitude) + ' ' + str(location_osm.latitude))
    if location_ya:
        task.loc[index, 'addr_ya'] = location_ya.address
        task.loc[index, 'lon_ya'] = location_ya.longitude
        task.loc[index, 'lat_ya'] = location_ya.latitude
        # print(location_ya.address)
        # print(str(location_ya.longitude) + ' ' + str(location_ya.latitude))

task.to_csv('../data/geocoded_achinsk.csv')